In [10]:
#importy
import numpy as np
import deepxde as dde


In [11]:
#stale
L = 2

In [12]:
#dziedzina
# geometry = dde.geometry.Hypercube(xmin=[-L/2, -L/2, 1, 1], xmax=[L/2, L/2, 2, 2])
g1 = dde.geometry.Hypercube(xmin=[-L/2, -L/2, 1, 1], xmax=[L/2, L/2, 2, 2])
g2 = dde.geometry.Hypercube(xmin=[-L/2, -L/2, 1, 1], xmax=[L/2, L/2, 1.01, 1.01])
geometry1 = dde.geometry.CSGIntersection(g1, g2)

g2 = dde.geometry.Hypercube(xmin=[-L/2, -L/2, 1.99, 1.99], xmax=[L/2, L/2, 2, 2])
geometry2 = dde.geometry.CSGIntersection(g1, g2)

geometry = dde.geometry.CSGUnion(geometry1, geometry2)

print(geometry.random_points(10))



[[-0.25768653 -0.6313861   1.9981074   1.9915226 ]
 [ 0.05297974  0.18608065  1.9903694   1.9913912 ]
 [ 0.81975126 -0.37570325  1.9909556   1.9908108 ]
 [ 0.56753445  0.14737682  1.9970841   1.9951679 ]
 [-0.28864774 -0.7577283   1.9939516   1.9976153 ]
 [ 0.11636163 -0.435433    1.0012335   1.0002722 ]
 [-0.35065842  0.60236555  1.9928739   1.9905862 ]
 [-0.33972523 -0.20446621  1.0072254   1.0058284 ]
 [-0.57374066 -0.76799184  1.9947776   1.9965396 ]
 [-0.08874463 -0.5267332   1.0000403   1.0012184 ]]


In [13]:
#funkcja
def pde(x, y):
    du_xx = dde.grad.hessian(y, x, i=0, j=0)
    du_yy = dde.grad.hessian(y, x, i=1, j=1)
    n1 = x[:, 2:3]
    n2 = x[:, 3:4]
    return 0.5 * (du_xx + du_yy) + (n1*n1 + n2*n2) * np.pi * y / (2 * L * L)



In [14]:
#warunki brzegowe
def boundaryFunction(x, on_boundary):
    if not on_boundary:
        return False
    if dde.utils.isclose(x[0], -L/2) or dde.utils.isclose(x[0], L/2):
        return True
    if dde.utils.isclose(x[1], -L/2) or dde.utils.isclose(x[1], L/2):
        return True
    return False


boundary_conditions = dde.icbc.DirichletBC(geometry, 
                                            lambda x: 0, 
                                            lambda x, on_boundary: boundaryFunction(x, on_boundary))


In [15]:
#dane
data = dde.data.PDE(geometry, 
                    pde, 
                    boundary_conditions, 
                    num_domain=4000,
                    num_boundary=2000,
                    num_test=1000
                    )

In [16]:
#siec
net = dde.nn.FNN([4]+[30]*3+[1],
                "tanh",
                "Glorot normal"
                )

In [17]:
#model
model = dde.Model(data, net)

model.compile("adam", lr=0.001)

Compiling model...
Building feed-forward neural network...
'build' took 0.088545 s

'compile' took 0.677119 s



In [18]:
#trenowanie
loss_history, train_state = model.train(iterations = 10000)

Training model...

Step      Train loss              Test loss               Test metric
0         [1.08e+00, 1.90e-01]    [1.04e+00, 1.90e-01]    []  
1000      [2.93e-05, 1.16e-05]    [2.35e-05, 1.16e-05]    []  
2000      [6.24e-06, 2.07e-06]    [5.12e-06, 2.07e-06]    []  
3000      [4.23e-06, 1.18e-06]    [3.48e-06, 1.18e-06]    []  
4000      [2.86e-06, 7.42e-07]    [2.37e-06, 7.42e-07]    []  
5000      [2.13e-06, 4.99e-07]    [1.81e-06, 4.99e-07]    []  
6000      [1.06e-06, 2.65e-07]    [8.69e-07, 2.65e-07]    []  
7000      [1.52e-03, 2.29e-04]    [1.50e-03, 2.29e-04]    []  
8000      [3.69e-07, 9.59e-08]    [2.91e-07, 9.59e-08]    []  
9000      [3.88e-07, 1.65e-07]    [3.45e-07, 1.65e-07]    []  
10000     [1.14e-07, 4.43e-08]    [8.83e-08, 4.43e-08]    []  

Best model at step 10000:
  train loss: 1.59e-07
  test loss: 1.33e-07
  test metric: []

'train' took 492.039620 s

